# 在 Azure AI 和 Azure ML 中结合使用 LangChain 与 Llama2

你可以将部署在 Azure AI 和 Azure ML 中的 Llama2 模型与`langchain`结合使用，以创建更复杂的智能应用程序。搭配使用`langchain_community`包与 Azure 机器学习集成。 

## 先决条件

在开始之前，我们需要采取一些特定步骤来部署模型：

* 注册一个有效的带有订阅的Azure账户
* 确保你有访问 [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home) 的权限
* 创建工程和资源组
* 从模型目录中选择Llama模型。本示例假设您正在部署`LLama-2-70b-chat`。

    > 请注意，某些模型可能并非在Azure AI和Azure机器学习的所有区域都可用。在这种情况下，你可以在模型可用的区域创建一个工作区或项目，然后通过来自其他区域的连接来使用它。若要详细了解如何使用连接，请参阅 [使用连接使用模型](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)。 

* 采用“即用即付”方式进行部署

一旦成功部署，您将获得用于推理的API端点和安全密钥。

有关更多信息，您应查阅Azure的官方文档[此处](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-llama)，了解模型部署和推理相关内容。

要完成本教程，你需要：

* 安装 `langchain` 和 `langchain_community`:

    ```bash
    pip install langchain langchain_community
    ```

## 示例

以下示例展示了如何创建一个链，该链使用部署在 Azure AI 和 Azure ML 中的 Llama2 聊天模型。该链已配置了`ConversationBufferMemory`。此示例改编自 [LangChain official documentation](https://python.langchain.com/docs/modules/memory/adding_memory).

In [ ]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_community.chat_models.azureml_endpoint import (
    AzureMLChatOnlineEndpoint,
    AzureMLEndpointApiType,
    LlamaChatContentFormatter,
)

让我们创建一个`AzureMLChatOnlineEndpoint`模型实例。这个类使我们能够访问部署在Azure AI或Azure ML中的任何模型。对于完成模型，使用`langchain_community.llms.azureml_endpoint.AzureMLOnlineEndpoint`类，并将`LlamaContentFormatter`作为`content_formatter`。

In [ ]:
chat_model = AzureMLChatOnlineEndpoint(
    endpoint_url="https://<endpoint-name>.<region>.inference.ai.azure.com/v1/chat/completions",
    endpoint_api_type=AzureMLEndpointApiType.serverless,
    endpoint_api_key="<key>",
    content_formatter=LlamaChatContentFormatter(),
)

> 提示：你可以配置环境变量`AZUREML_ENDPOINT_URL`、`AZUREML_ENDPOINT_API_KEY`和`AZUREML_ENDPOINT_API_TYPE`，而不是将它们作为参数传递。

在以下提示中，我们有两个输入键：一个用于实际输入（`human_input`），另一个用于来自`Memory`类的输入（`chat_history`）。

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human. You love making references to animals on your answers."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

我们按如下方式创建链：

In [ ]:
chat_llm_chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    memory=memory,
    verbose=True,
)

我们可以看到它是如何运作的：

In [ ]:
chat_llm_chain.predict(human_input="Hi there my friend")

In [ ]:
chat_llm_chain.predict(
    human_input="I'm thinking on a present for my mother. Any advise?"
)

## 更多资源

下面是一些额外的参考资料:  

* [Fine-tune Llama](https://learn.microsoft.com/azure/ai-studio/how-to/fine-tune-model-llama)
* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
